In [1]:
#Importing Libraries
import nltk
from nltk.stem.lancaster import LancasterStemmer
from PIL import Image as pil
import tflearn
import tensorflow as tf
tf.compat.v1.disable_v2_behavior
import numpy as np
import random
import json
import pickle

stemmer = LancasterStemmer()


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)



In [2]:
#Unpickling data
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

#Importing chatbot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
#Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

#Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



In [6]:
def clean_up_sentence(sentence):
    #Tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    #Stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return: 0(absence) or 1(presence) for each word in the bag that exists in the sentence
def words_presence(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [9]:
#p = words_presence("when does your shop open today?", words)
#print (p)
p = words_presence("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 1 0]
['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']


In [10]:
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from c:\Users\Diya\OneDrive\Desktop\Programs\Chatbot\Chatbot\model.tflearn


In [11]:
#Store context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    #Generating probabilities from the model
    results = model.predict([words_presence(sentence, words)])[0]
    #Filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    #Sorting by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))

    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    #If there is a classification then finding the matching intent tag
    if results:
        while results:
            for i in intents['intents']:
                #Finding a tag matching the first result
                if i['tag'] == results[0][0]:
                    #Setting context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    #Checking if this intent is contextual and applies to the user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [22]:
#Testing classify
classify('is your shop open today?')
#classify('When does your shop open?')

[('opentoday', 0.8625594)]

In [30]:
#Testing Response
response('is your shop open today?')
response('when do you open?')

We're open every day from 9am-9pm
Our hours are 9am-9pm every day


In [40]:
response('what kind of mopeds do you rent?')

We rent Yamaha, Piaggio and Vespa mopeds


In [38]:
response('Goodbye, see you later')

Bye! Come back again soon.


In [43]:
context

{}

In [55]:
response('we want to rent a moped')

Are you looking to rent today or later this week?


In [54]:
context

{'123': 'rentalday'}

In [58]:
response("Hi there!", show_details=True)

context: 
tag: greeting
Good to see you again
